### **기본 주행 제어**

In [ ]:
#!/usr/bin/env python3
import rospy
from fs_msgs.msg import ControlCommand

def send_command():
    # 노드 초기화
    rospy.init_node('simple_control_publisher', anonymous=True)

    # 퍼블리셔 생성
    pub = rospy.Publisher('/fsds/control_command', ControlCommand, queue_size=1)

    # 10Hz (초당 10번)
    rate = rospy.Rate(10)

    # 메시지 생성
    cmd = ControlCommand()
    cmd.throttle = 0.3
    cmd.steering = -0.5
    cmd.brake = 0.0

    rospy.loginfo("Publishing control command at 10Hz...")

    # 계속 퍼블리시
    while not rospy.is_shutdown():
        pub.publish(cmd)
        rate.sleep()

if __name__ == "__main__":
    try:
        send_command()
    except rospy.ROSInterruptException:
        pass

### **LiDAR 기반 자동 정지 주행 (정지 후 다시 주행)**


*   장애물이 감지되면 멈췄다가, 사라지면 다시 출발하는 동작을 수행








In [ ]:
#!/usr/bin/env python3
import rospy
from sensor_msgs.msg import PointCloud2
from fs_msgs.msg import ControlCommand
import sensor_msgs.point_cloud2 as pc2
import numpy as np

# 전역 변수
pub = None

def lidar_callback(data):
    global pub

    # LiDAR 포인트 읽기
    points = np.array(list(pc2.read_points(data, field_names=("x", "y", "z"), skip_nans=True)))
    if len(points) == 0:
        return

    # 전방 10m, 좌우 2m 이내 포인트만 필터링
    front_points = points[
        (points[:, 0] > 0) & (points[:, 0] < 10.0) &
        (np.abs(points[:, 1]) < 2.0)
    ]

    # 거리 계산 (가장 가까운 장애물까지)
    min_distance = np.inf
    if len(front_points) > 0:
        min_distance = np.min(np.linalg.norm(front_points[:, :2], axis=1))

    cmd = ControlCommand()
    cmd.steering = 0.0

    # 장애물 3m 이내 접근 시 정지
    if min_distance < 3.0:
        rospy.logwarn(f" 장애물 감지! 거리={min_distance:.2f} m → 차량 정지.")
        cmd.throttle = 0.0
        cmd.brake = 1.0
    else:
        cmd.throttle = 0.4
        cmd.brake = 0.0

    pub.publish(cmd)

def main():
    global pub
    rospy.init_node('simple_lidar_stop_node')
    pub = rospy.Publisher('/fsds/control_command', ControlCommand, queue_size=10)
    rospy.Subscriber('/fsds/lidar/Lidar1', PointCloud2, lidar_callback)
    rospy.loginfo("출발!")
    rospy.spin()

if __name__ == '__main__':
    main()


### **LiDAR 기반 자동 정지 주행 (정지 후 정지 유지)**


*   장애물이 감지되면 멈추고, 이후 계속 정지 상태를 유지



In [ ]:
#!/usr/bin/env python3
import rospy
from sensor_msgs.msg import PointCloud2
from fs_msgs.msg import ControlCommand
import sensor_msgs.point_cloud2 as pc2
import numpy as np

# -----------------------------
# 전역 변수
# -----------------------------
pub = None
state = "DRIVE"  # 상태: DRIVE / STOP

def lidar_callback(data):
    global pub, state

    # LiDAR 포인트 읽기
    points = np.array(list(pc2.read_points(data, field_names=("x", "y", "z"), skip_nans=True)))
    if len(points) == 0:
        return

    # 전방 10m, 좌우 2m 범위 포인트 필터
    front_points = points[
        (points[:, 0] > 0) & (points[:, 0] < 10.0) &
        (np.abs(points[:, 1]) < 2.0)
    ]

    # 가장 가까운 장애물까지 거리 계산
    min_distance = np.inf
    if len(front_points) > 0:
        min_distance = np.min(np.linalg.norm(front_points[:, :2], axis=1))

    cmd = ControlCommand()
    cmd.steering = 0.0

    # -----------------------------
    # 상태 1: 주행 중
    # -----------------------------
    if state == "DRIVE":
        if min_distance < 2.0:
            rospy.logwarn(f"장애물 감지! 거리={min_distance:.2f} m → 차량 정지.")
            state = "STOP"
            cmd.throttle = 0.0
            cmd.brake = 1.0
        else:
            cmd.throttle = 0.4
            cmd.brake = 0.0

    # -----------------------------
    # 상태 2: 정지 유지
    # -----------------------------
    elif state == "STOP":
        cmd.throttle = 0.0
        cmd.brake = 1.0
        rospy.loginfo_once("차량이 정지 상태를 유지합니다.")

    pub.publish(cmd)

def main():
    global pub
    rospy.init_node('lidar_stop_hold_node')
    pub = rospy.Publisher('/fsds/control_command', ControlCommand, queue_size=10)
    rospy.Subscriber('/fsds/lidar/Lidar1', PointCloud2, lidar_callback)
    rospy.loginfo("출발!")
    rospy.spin()

if __name__ == '__main__':
    main()


### **차량 정지 명령 (수동 제어)**

In [ ]:
rostopic pub -1 /fsds/control_command fs_msgs/ControlCommand "{throttle: 0.0, steering: 0.0, brake: 1.0}"

### **시뮬레이터 리셋 명령**

In [ ]:
rosservice call /fsds/reset "{waitOnLastTask: true}"